In [1]:
from IPython.display import display, HTML

## 3.Setting up the board

### Naming conventions
#### Rows and Columns
- Rows will be labelled by the letters A,B,C,D,E,F,G,H
- Columns will be labelled by the numbers 1,2,3,4,5,6,7,8,9

#### Boxes, Units and Peers
- **boxes**: individual squares ex. `'A1,'A2'...'I9'`
- **units**: complete rows, columns, and $3x3$ squares
- **peers**: all the other boxes that belong to a common unit ie peers of`A1'..

## 4. Encoding the Board"

In [ ]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [ ]:
def cross(a,b):
    return[s+t for s in a for t in b]

In [ ]:
boxes = cross(rows, cols)

In [ ]:
row_units = [cross(r,cols) for r in rows]

column_units = [cross(rows,c) for c in cols]

square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]


unitlist = row_units + column_units + square_units

In [ ]:
units = dict((s, [u for u in unitlist if s in u])for s in boxes)

peers = dict((s, set(sum(units[s], []))- set([s])) for s in boxes)

In [ ]:
def display(values):
    '''
    Display the values as 2D grid.
    input: The sodoku in dictionary form
    output: None
    '''
    width = 1+ max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r + c].center(width) + ('|' if c in '36' else '')
                     for c in cols))
        if r in 'CF': 
            print(line)
    return      

### `grid_values()`

In [ ]:
def grid_values(grid):
    '''
    convert grid string into {<box>: <val>} dict with '.' values for emplties
    
    args:
        grid: Sudoku grid in string form, 81 chars long
    returns:
        Sudou grid indict form:
            - key: Box labels, eg 'A1'
            - val: value in correstonidgn box, e.g '8', or '.' if empty.
    '''
    
    assert len(grid) == 81 # input grid must be a string of length 81 (9x9)
    return dict(zip(boxes, grid))

#### TEST

In [ ]:
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))


---
## 5 strategy 1. Emimination
#### If a box has a value assigned, then non of the peers of this box can have this value

In [4]:
display(HTML('''
<img src="images/reduce-values.png"   align=left width="41%">
<img src="images/values-easy.png"   align=left width="41%">
'''))

#### CODE
new `grid_values()` function that changes the empty slot into a string of `'123456789'`

In [ ]:
def grid_values(grid):
    '''
    Convert grid string into {<box>: <value>} dict with '123456789' values for empties
    
    args:
        grid: Sudoku grid in string form, 81 chars long
    returns:
        sudoku grid in dic form:
        - key: box labels ex. 'A1
        - val: Value in correspinind box, ex. '8' or '123456789' if it is empty.
    '''
    values = []
    all_digits = '123456789'
    for c in grid:
        if c =='.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))
    


#### TEST

In [ ]:
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))


In [ ]:
solution = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
solution

### Imlement `eliminate()`
Now we will implement eliminate

In [ ]:
def eliminate(values):
    '''
    Eliminate values from peers of each box with a single value.
    
    Go throug all the boxes, and whenever there is a box with a single value,
    eliminate this value form the set of values of all its peers.
    
    args:
        values: sudoku in dict form
    returs:
        resulting sudoju in dict form after eliminating values.
    '''
    
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit, '')
    return values

#### TEST

In [ ]:
eliminate(solution)

#### oooorrr,,
```python
{
 'A1': '45','A2': '4578','A3': '3','A4': '49','A5': '2','A6': '147','A7': '6','A8': '5789','A9': '57',
 'B1': '9','B2': '24678','B3': '47','B4': '3','B5': '47','B6': '5','B7': '78','B8': '278','B9': '1',
 'C1': '25','C2': '257','C3': '1','C4': '8','C5': '79','C6': '6','C7': '4','C8': '23579','C9': '2357',
 'D1': '345','D2': '345','D3': '8','D4': '1','D5': '3456','D6': '2','D7': '9','D8': '34567','D9': '34567',
 'E1': '7','E2': '123459','E3': '49','E4': '459','E5': '34569','E6': '4','E7': '1','E8': '13456','E9': '8',
 'F1': '1345','F2': '13459','F3': '6','F4': '7','F5': '3459','F6': '8','F7': '2','F8': '1345','F9': '345',
 'G1': '134','G2': '1347','G3': '2','G4': '6','G5': '478','G6': '9','G7': '5','G8': '1478','G9': '47',
 'H1': '8','H2': '1467','H3': '47','H4': '2','H5': '457','H6': '3','H7': '17','H8': '1467','H9': '9',
 'I1': '46','I2': '4679','I3': '5','I4': '4','I5': '1','I6': '47','I7': '3','I8': '24678','I9': '2467'
 }

```

In [5]:
display(HTML('''
<img src="images/values-easy.png"   align=left width="41%">
'''))

## 6. Strategy 2: Only Choice

In [7]:
display(HTML('''
<img src="images/values-easy.png"   align=left width="40%">
<img src="images/highlighted-unit.png"   align=left width="40%">
'''))

### Strategy 2: Only Choice